In [71]:
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default='notebook'

import random
import tqdm
import warnings
import plotly
import plotly.graph_objects as go

from scipy.stats import spearmanr
from plotly.subplots import make_subplots
from collections import Counter

DATA = "../data/experiment_2"

MODELS = [
  "gpt3_rating",
  "gpt4_rating",
  "gpt3_scm",
  "gpt4_scm",
  "human_scm"
]

TOP_PROMPT_GENERATOR = "S3-C1-A1-Q1-O1-T"

np.random.seed(1)
random.seed(1)

NUM_PARTICIPANTS = 610

# Load experiment 2 participant ratings

In [72]:
human_df = pd.read_csv(f"{DATA}/clean_human_ratings.csv", index_col=0)

# Filter out control and tutorial arguments
human_df = human_df[~human_df["is_control"]].dropna().rename({"argument": "argument_id"}, axis=1)
human_df["ranking"] = human_df.groupby(["pid", "is_single_premise"])["rating"].rank(pct="True", ascending=True)

# Map participants to blocks based on the multi premise arguments that they saw
blocks = {}
block_splits = {}
for pid, pid_df in human_df[~human_df["is_single_premise"]].groupby(["pid"]):
    block = tuple(pid_df["argument_id"].sort_values().tolist())
    if block not in blocks:
        blocks[block] = [pid]
    else:
        blocks[block].append(pid)
        
    bs = (pid_df.iloc[0]["domain"], pid_df.iloc[0]["conclusion_type"])
    if bs not in block_splits:
        block_splits[bs] = []
    block_splits[bs].append(block)
    
pid_to_block = {}
for i,pids in enumerate(blocks.values()):
    for pid in pids:
        pid_to_block[pid] = i
human_df["block"] = human_df["pid"].map(pid_to_block)

assert len(human_df["pid"].unique()) == NUM_PARTICIPANTS
assert len(pid_to_block) == NUM_PARTICIPANTS
assert len(blocks) == 61
assert all(len(set(v))== 11 if k == ('Birds', 'General') else len(set(v)) == 10 for k,v in block_splits.items())
assert all(len(set(v)) == 10 for v in blocks.values())

human_df.head()

,pid,tid,argument_id,domain,conclusion_type,is_single_premise,is_control,premises,conclusion,rating,light_cut,medium_cut,hard_cut,ranking,block
0,0,0,"(('Eagles',), 'All birds')",Birds,General,True,False,"('Eagles',)",All birds,0,False,False,False,0.041667,0
1,0,1,"(('Crows',), 'All birds')",Birds,General,True,False,"('Crows',)",All birds,7,False,False,False,0.583333,0
2,0,10,"(('Vultures',), 'All birds')",Birds,General,True,False,"('Vultures',)",All birds,3,False,False,False,0.187500,0
3,0,11,"(('Falcons',), 'All birds')",Birds,General,True,False,"('Falcons',)",All birds,3,False,False,False,0.187500,0
4,0,12,"(('Herons',), 'All birds')",Birds,General,True,False,"('Herons',)",All birds,7,False,False,False,0.583333,0


In [73]:
# Check our blocks
argument_block_map = {}
for split in block_splits:
    for block in block_splits[split]:
        for argument in block:
            if argument in argument_block_map:
                argument_block_map[argument].add(block)
            else:
                argument_block_map[argument] = set([block])
            
for argument, argument_blocks in argument_block_map.items():
    if len(set(argument_blocks)) > 1:
        print(argument)

(('Sparrows', 'Robins'), 'All birds')
(('Magpies', 'Ducks'), 'All birds')
(('Turkeys', 'Roosters'), 'All birds')
(('Seagulls', 'Ducks'), 'All birds')
(('Chickens', 'Magpies'), 'All birds')
(('Herons', 'Eagles'), 'All birds')
(('Sparrows', 'Ducks'), 'All birds')
(('Falcons', 'Vultures'), 'All birds')
(('Swans', 'Parrots'), 'All birds')
(('Swans', 'Sparrows'), 'All birds')


Some bird arguments appear in more than one block because we had 11 blocks for general birds rather than 10.

# Load experiment 2 model ratings

In [74]:
model_df = pd.read_csv(f"{DATA}/all_ratings.csv", index_col=0)
model_df = model_df[model_df["prompt_generator"] == TOP_PROMPT_GENERATOR]
model_df = model_df[["argument_id"] + MODELS].reset_index(drop=True)

# Add in split labels to model_df
cols = model_df.columns.tolist()[1:]
s = model_df.shape[0]
model_df = model_df.merge(human_df[["argument_id","domain","conclusion_type","is_single_premise"]].drop_duplicates(), on="argument_id")
cols = ["argument_id","domain","conclusion_type","is_single_premise"] + cols
model_df = model_df[cols]

assert set(human_df["argument_id"]) == set(model_df["argument_id"])
assert len(set(model_df["argument_id"])) == model_df.shape[0]

model_df.head()

,argument_id,domain,conclusion_type,is_single_premise,gpt3_rating,gpt4_rating,gpt3_scm,gpt4_scm,human_scm
0,"(('Airplanes', 'Buses'), 'All vehicles')",Vehicles,General,False,63.913538,70.0,0.652174,0.727053,0.758303
1,"(('Airplanes', 'Helicopters'), 'All vehicles')",Vehicles,General,False,38.774614,55.0,0.443690,0.520380,0.519173
2,"(('Airplanes', 'Taxis'), 'All vehicles')",Vehicles,General,False,65.719333,70.0,0.572917,0.719354,0.750755
3,"(('Airplanes', 'Trams'), 'Trains')",Vehicles,Specific,False,90.729240,85.0,0.819444,0.818765,0.816501
4,"(('Airplanes', 'Zeppelins'), 'All vehicles')",Vehicles,General,False,30.386649,50.0,0.563406,0.510417,0.534118


# Bootstrap model correlations

In [75]:
ITERS = 1000

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    rows = []
    for i in tqdm.tqdm(range(ITERS)):

        # Sample N participants with replacement from each block
        pid_sample = []
        for block_pids in blocks.values():
            
            block_sample = random.choices(block_pids, k=len(block_pids))
            
            assert len(block_sample) == len(block_pids)
            
            pid_sample += block_sample
        
        assert len(pid_sample) == NUM_PARTICIPANTS

        # Construct a new bootstrapped 'human_df' dataframe from the above sample
        tdf = pd.DataFrame([], columns=human_df.columns)
        for ps in pid_sample:
            ptdf = human_df[human_df["pid"] == ps]
            tdf = pd.concat([tdf, ptdf])
        
        assert tdf.shape[0] == human_df.shape[0]
            
        # Construct a new aggregated human ranking dataframe from the above dataframe
        sample_human_df = pd.DataFrame({"sample_human_ranking": tdf[["argument_id", "ranking"]].groupby("argument_id")["ranking"].mean()}).reset_index()
        
        assert sample_human_df.shape[0] == len(set(sample_human_df["argument_id"]))
        
        # Compare model ratings/rankings to sample human rankings
        for g, gdf in model_df.groupby(["domain", "conclusion_type", "is_single_premise"]):
            domain, conclusion_type, argument_type = g
            
            sample_human_model_df = gdf.merge(sample_human_df, on="argument_id")
            
            assert sample_human_model_df.shape[0] == gdf.shape[0]

            # Calculate Spearman R for all models
            statistics, pvalues = [], []
            for model in MODELS:
                spearman = spearmanr(
                    sample_human_model_df["sample_human_ranking"].tolist(), 
                    sample_human_model_df[model].tolist()
                )
                
                statistics.append(spearman.correlation)
                pvalues.append(spearman.pvalue)

            rows.append((i, domain, conclusion_type, argument_type) + tuple(statistics) + tuple(pvalues))

bootstrap_df = pd.DataFrame(rows, columns=["i", "domain", "conclusion_type", "is_single_premise"] + [f"{r}_spearmanr" for r in MODELS] + [f"{r}_pval" for r in MODELS])
bootstrap_df = bootstrap_df.rename({c: f'{c.replace("_rating", "")}' for c in bootstrap_df.columns}, axis=1)
bootstrap_df.to_csv(f"{DATA}/results/blockwise_participant_bootstrap.csv")

100%|██████████| 1000/1000 [52:50<00:00,  3.17s/it]


In [76]:
bootstrap_df = pd.read_csv(f"{DATA}/results/blockwise_participant_bootstrap.csv", index_col=0)

rows = []
for g, gdf in tqdm.tqdm(bootstrap_df.groupby(["domain", "conclusion_type", "is_single_premise"])):
    means, ses = [], []
    for model in MODELS:
        model = model.replace("_rating", "")
        means.append(np.mean(gdf[f"{model}_spearmanr"]))
        ses.append(np.std(gdf[f"{model}_spearmanr"], ddof=1))
    rows.append(g + tuple(means) + tuple(ses))

bootstrap_summary_df = pd.DataFrame(rows, columns=["domain", "conclusion_type", "is_single_premise"] + [f"{m}_mean" for m in MODELS] + [f"{m}_se" for m in MODELS])
bootstrap_summary_df = bootstrap_summary_df.rename({c: f'{c.replace("_rating", "")}_spearmanr' if "gpt" in c or "scm" in c else c for c in bootstrap_summary_df.columns}, axis=1)
bootstrap_summary_df.to_csv(f"{DATA}/results/blockwise_participant_bootstrap_summary.csv")
bootstrap_summary_df

100%|██████████| 12/12 [00:00<00:00, 1200.03it/s]


,domain,conclusion_type,is_single_premise,gpt3_mean_spearmanr,gpt4_mean_spearmanr,gpt3_scm_mean_spearmanr,gpt4_scm_mean_spearmanr,human_scm_mean_spearmanr,gpt3_se_spearmanr,gpt4_se_spearmanr,gpt3_scm_se_spearmanr,gpt4_scm_se_spearmanr,human_scm_se_spearmanr
0,Birds,General,False,0.307819,0.233067,0.050022,0.233026,0.060316,0.073866,0.072141,0.057300,0.074178,0.085654
1,Birds,General,True,0.495856,0.364424,0.454952,0.616570,0.091167,0.073886,0.075477,0.057166,0.072710,0.063616
2,Birds,Specific,False,0.364529,0.265468,0.496416,0.499215,0.431251,0.061541,0.056917,0.063322,0.061136,0.068518
3,Birds,Specific,True,0.425218,0.417392,0.497358,0.490772,0.540120,0.029536,0.033968,0.033780,0.039947,0.034482
4,Mammals,General,False,0.316019,0.183663,0.257143,0.277945,0.332384,0.073938,0.079834,0.064132,0.070442,0.077745
5,Mammals,General,True,0.273085,0.010921,0.612598,0.696820,0.594731,0.067645,0.086875,0.054802,0.060555,0.065780
6,Mammals,Specific,False,0.602578,0.740621,0.762699,0.765068,0.801509,0.027700,0.028062,0.030151,0.031305,0.028949
7,Mammals,Specific,True,0.594100,0.706128,0.699561,0.747253,0.810142,0.020942,0.019947,0.018289,0.020290,0.019940
8,Vehicles,General,False,0.556656,0.395466,0.369142,0.406362,0.522385,0.060729,0.053427,0.062749,0.056183,0.057921
9,Vehicles,General,True,0.863714,0.640721,0.439150,0.608896,0.862359,0.017377,0.020899,0.039567,0.031996,0.020767


# Calculate Split-half Reliability

In [77]:
ITERS = 1000

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    rows = []
    for i in tqdm.tqdm(range(ITERS)):
        for split, split_df in human_df.groupby(["domain", "conclusion_type", "is_single_premise"]):
            domain, conclusion_type, is_single_premise = split
            a_block_ratings = pd.DataFrame([], columns=split_df.columns)
            b_block_ratings = pd.DataFrame([], columns=split_df.columns)
    
            # Split each block of participants in two
            for block, block_df in split_df.groupby("block"):

                # Randomly divide this block's participants into sets A and B
                block_pids = list(block_df["pid"].unique())
                a_pids = random.sample(block_pids, k=len(block_pids)//2)
                b_pids = [bp for bp in block_pids if bp not in a_pids]
                a_block_df = block_df[block_df["pid"].isin(a_pids)]
                b_block_df = block_df[block_df["pid"].isin(b_pids)]

                a_block_ratings = pd.concat([a_block_ratings, a_block_df]).reset_index(drop=True)
                b_block_ratings = pd.concat([b_block_ratings, b_block_df]).reset_index(drop=True)
                
                assert set(a_block_df["argument_id"]) == set(b_block_df["argument_id"])
                assert a_block_df.shape[0] + b_block_df.shape[0] == block_df.shape[0]
                
                # Each argument in A and B can have a difference of at most 1 participant, so the difference in the total number of participants should be no more than the max number of arguments
                assert abs(a_block_df.shape[0] - b_block_df.shape[0]) <= max(len(a_block_df["argument_id"].unique()), len(b_block_df["argument_id"].unique()))
            
            assert set(a_block_ratings["argument_id"]) == set(b_block_ratings["argument_id"])
            assert a_block_ratings.shape[0] + b_block_ratings.shape[0] == split_df.shape[0]
            
            # Each argument in A and B can have a difference of at most 1 participant, so the difference in the total number of participants should be no more than the max number of arguments
            assert abs(a_block_ratings.shape[0] - b_block_ratings.shape[0]) <= max(len(a_block_ratings["argument_id"].unique()), len(b_block_ratings["argument_id"].unique()))
            
            # Calculated mean argument ranking for A and B
            a_ratings = pd.DataFrame({"a_ranking": a_block_ratings.groupby(["argument_id", "domain", "conclusion_type", "is_single_premise"])["ranking"].mean()}).reset_index()
            b_ratings = pd.DataFrame({"b_ranking": b_block_ratings.groupby(["argument_id", "domain", "conclusion_type", "is_single_premise"])["ranking"].mean()}).reset_index()
            
            assert set(a_ratings["argument_id"]) == set(a_block_ratings["argument_id"])
            assert set(b_ratings["argument_id"]) == set(b_block_ratings["argument_id"])
            
            # Calculate number of participants per argument in A and B
            a_num_participants = pd.DataFrame({"num_participants": a_block_ratings.groupby("argument_id").size()}).reset_index()
            b_num_participants = pd.DataFrame({"num_participants": b_block_ratings.groupby("argument_id").size()}).reset_index()
            a_ratings = a_ratings.merge(a_num_participants, on="argument_id")
            b_ratings = b_ratings.merge(b_num_participants, on="argument_id")
            
            # Ratings dataframes should contain no duplicates and all the right arguments
            for x_ratings in [a_ratings, b_ratings]:
                assert x_ratings.drop_duplicates().shape[0] == x_ratings.shape[0]
                assert x_ratings.shape[0] == len(set(x_ratings["argument_id"]))
                assert set(x_ratings["argument_id"]) == set(split_df["argument_id"])
                
            # Number of participants for each argument in samples a and b should be roughly the same
            num_ratings = a_ratings[["argument_id", "num_participants"]].merge(b_ratings[["argument_id", "num_participants"]], on="argument_id")
            num_ratings["num_diff"] = abs(num_ratings["num_participants_x"] - num_ratings["num_participants_y"])
            assert all(x <= 1 for x in num_ratings["num_diff"])
            assert a_ratings["num_participants"].sum() + b_ratings["num_participants"].sum() == split_df.shape[0]
            
            # Calculate correlation between A and B rankings
            t_ratings_df = a_ratings.merge(b_ratings, on=["argument_id", "domain", "conclusion_type", "is_single_premise"])
            ss =  spearmanr(t_ratings_df["a_ranking"], t_ratings_df["b_ranking"])
            rows.append((i,domain,conclusion_type,is_single_premise,ss.correlation,ss.pvalue))

ratings_df = pd.DataFrame(rows, columns=["i", "domain", "conclusion_type", "is_single_premise", "statistic", "pvalue"])
ratings_df.to_csv(f"{DATA}/results/blockwise_split_half_reliabilities.csv")

100%|██████████| 1000/1000 [13:18<00:00,  1.25it/s]


In [78]:
ratings_df = pd.read_csv(f"{DATA}/results/blockwise_split_half_reliabilities.csv", index_col=0)

rows = []
for g, gdf in tqdm.tqdm(ratings_df.groupby(["domain", "conclusion_type", "is_single_premise"])):
    rows.append(g + (gdf["statistic"].mean(), gdf["statistic"].std()))
summary_ratings_df = pd.DataFrame(rows, columns=["domain", "conclusion_type", "is_single_premise", "spearmanr_mean", "spearmanr_std"])

summary_ratings_df.to_csv(f"{DATA}/results/blockwise_split_half_reliabilities_summary.csv")
summary_ratings_df

100%|██████████| 12/12 [00:00<00:00, 2400.06it/s]


,domain,conclusion_type,is_single_premise,spearmanr_mean,spearmanr_std
0,Birds,General,False,0.115118,0.081461
1,Birds,General,True,0.733808,0.081343
2,Birds,Specific,False,0.326532,0.070603
3,Birds,Specific,True,0.562417,0.038141
4,Mammals,General,False,0.161668,0.084412
5,Mammals,General,True,0.755261,0.068096
6,Mammals,Specific,False,0.765347,0.030081
7,Mammals,Specific,True,0.804185,0.019442
8,Vehicles,General,False,0.372382,0.065552
9,Vehicles,General,True,0.941449,0.017990


# Plot bootstrapped model correlations + split half reliability

In [84]:
SCALE_FACTOR = 1

summary_ratings_df = pd.read_csv(f"{DATA}/results/blockwise_split_half_reliabilities_summary.csv", index_col=0)
summary_ratings_df["argument_type"] = summary_ratings_df["is_single_premise"].apply(lambda x: "single" if x else "multi")

bootstrap_summary_df = pd.read_csv(f"{DATA}/results/blockwise_participant_bootstrap_summary.csv", index_col=0)
bootstrap_summary_df["argument_type"] = bootstrap_summary_df["is_single_premise"].apply(lambda x: "single" if x else "multi")


GRID_COLOUR= "#e6e6e6"
SPLIT_RELIABILITY_COLOUR = "#cbc9ff"

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        '<b>Specific, Single Premise</b>', 
        '<b>General, Single Premise</b>', 
        '<b>Specific, Two Premise</b>', 
        '<b>General, Two Premise</b>'
    ],
    shared_yaxes=True,
    vertical_spacing=0.05,
    horizontal_spacing=0.02,
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'scatter'}]]
)

domain_mapping = {'Mammals': 1, 'Birds': 2, 'Vehicles': 3}
marker_colors = ['green', '#94d1a4', 'red', 'pink', 'blue']
offsets = [-0.2, -0.1, 0, 0.1, 0.2]
rating_columns = [
                    'gpt3_mean_spearmanr', 
                    'gpt3_scm_mean_spearmanr', 
                    'gpt4_mean_spearmanr', 
                    'gpt4_scm_mean_spearmanr', 
                    'human_scm_mean_spearmanr'
                 ]


def add_dumbbell_trace(fig, df, rating_col, marker_color, row, col, legend, offset):
    temp_df = df.copy()
    temp_df['x'] = temp_df['domain'].map(domain_mapping) + offset
    fig.add_trace(go.Scatter(
        x=temp_df['x'],
        y=temp_df[rating_col],
        error_y=dict(type='data', array=df[rating_col.replace("mean", "se")], symmetric=True, visible=True, width=7*SCALE_FACTOR, thickness=5*SCALE_FACTOR, color=marker_color),
        mode='markers',
        marker=dict(color=marker_color, size=20*SCALE_FACTOR),
        showlegend=legend,
        legendgroup=rating_col,
        name=rating_col.split('_')[0].replace("gpt", "GPT-") if 'scm' not in rating_col else ' '.join([rc for rc in rating_col.split('_')[:2] if rc != "corr"]).replace("gpt", "GPT-").replace("scm", "SCM"),
        customdata=temp_df['domain'],
        texttemplate="%{customdata}<br>%{y}",
        hovertemplate="%{customdata}<br>%{y}",
    ), row=row, col=col)


shapes = []
for i, (conclusion_type, argument_type) in enumerate([
        ('Specific', 'single'), 
        ('General', 'single'), 
        ('Specific', 'multi'), 
        ('General', 'multi')
    ], start=1):
    
    row, col = (i - 1) // 2 + 1, (i - 1) % 2 + 1
    data = bootstrap_summary_df[(bootstrap_summary_df['conclusion_type'] == conclusion_type) & (bootstrap_summary_df['argument_type'] == argument_type)]

    split_half_df = summary_ratings_df[(summary_ratings_df["conclusion_type"] == conclusion_type) & (summary_ratings_df["argument_type"] == argument_type)]
    split_half_df["x"] = split_half_df["domain"].map(domain_mapping)
    shapes = [
        dict(type="rect", xref=f"x{row if row > 1 else ''}", yref=f"y{col if col > 1 else ''}", x0=r["x"] - 0.25, y0=r["spearmanr_mean"] + r["spearmanr_std"], x1=r["x"] + 0.25, y1=r["spearmanr_mean"] - r["spearmanr_std"], fillcolor=SPLIT_RELIABILITY_COLOUR, line_width=0, layer="below")
        for _, r in split_half_df.iterrows()
    ]
    for s in shapes:
        fig.add_shape(s, row=row, col=col)

    for rating_col, marker_color, offset in zip(rating_columns, marker_colors, offsets):
        add_dumbbell_trace(fig, data, rating_col, marker_color, row, col, (i==1), offset)

fig.update_layout(
    width=1600*SCALE_FACTOR,
    height=800*SCALE_FACTOR, 
    legend=dict(orientation='v', yanchor='top', xanchor='right', x=1.13, font=dict(size=20*SCALE_FACTOR)),
    plot_bgcolor="#f5f6f7",
)

fig.update_annotations(font_size=24)

fig.update_yaxes(range=[-0.5, 1], matches='y')
fig.update_xaxes(tickvals=list(domain_mapping.values()), range=[0.5,3.5], ticktext=[f"{k}</br>" for k in list(domain_mapping.keys())])
for r in range(1,3):
    for c in range(1,3):
        fig.update_xaxes(showgrid=False, tickfont={"size": 20*SCALE_FACTOR}, tickmode="array", tickvals=[1,2,3], row=r, col=c, showticklabels=r==2)
        fig.update_yaxes(gridwidth=6*SCALE_FACTOR, zerolinewidth=6*SCALE_FACTOR, zerolinecolor=GRID_COLOUR, gridcolor=GRID_COLOUR, tickfont={"size": 20*SCALE_FACTOR}, tickmode="array", tickvals=[-0.5,0,0.5,1], ticktext=[f"{k} " for k in [-0.5,0,0.5,1]], row=r, col=c)

fig.show()

<ipython-input-84-ee5e1372d2ae>:70: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Compare model bootstraps

In [81]:
bootstrap_df = pd.read_csv(f"{DATA}/results/blockwise_participant_bootstrap.csv", index_col=0)

# In each split, calculate proportion of times that pairs of models beat each other
pairs = [
    ("gpt3", "gpt3_scm"),
    ("gpt3_scm", "gpt3"),
    ("gpt3", "gpt4"),
    ("gpt3_scm", "gpt4"),
    ("gpt4_scm", "gpt3"),
    ("gpt4", "gpt3"),
    ("gpt4", "gpt4_scm"),
    ("gpt4_scm", "gpt4"),
    ("gpt4", "human_scm"),
    ("gpt4_scm", "human_scm"),
    ("gpt3", "human_scm"),
    ("gpt3_scm", "human_scm"),
]

rows = []
for g, gdf in bootstrap_df.groupby(["domain", "conclusion_type", "is_single_premise"]):
    
    assert gdf.shape[0] == 1000
    
    outcomes = []
    for pair in pairs:
        p1, p2 = pair
        outcomes.append(gdf[gdf[f"{p2}_spearmanr"] > gdf[f"{p1}_spearmanr"]].shape[0] / gdf.shape[0])
    rows.append(g + tuple(outcomes))

bootstrap_outcome_df = pd.DataFrame(rows, columns=["domain", "conclusion_type", "is_single_premise"] + [f"{p2}_beats_{p1}" for p1,p2 in pairs])
bootstrap_outcome_df.to_csv(f"{DATA}/blockwise_participant_bootstrap_model_comparisons.csv")

In [82]:
# Generate latex table for paper
bootstrap_outcome_df["ct_order"] = bootstrap_outcome_df["conclusion_type"].map({"Specific": 0, "General": 1})
bootstrap_outcome_df["isp_order"] = bootstrap_outcome_df["is_single_premise"].map({True:0, False: 1})
bootstrap_outcome_df["d_order"] = bootstrap_outcome_df["domain"].map({"Mammals":0,"Birds":1,"Vehicles":2})

bootstrap_outcome_df = bootstrap_outcome_df.sort_values(by=["ct_order", "isp_order", "d_order"])
bootstrap_outcome_df["num_premises"] = bootstrap_outcome_df["is_single_premise"].map({True: "Single", False: "Multi"})
df = bootstrap_outcome_df[["conclusion_type", "num_premises", "domain", "human_scm_beats_gpt4", "human_scm_beats_gpt3", "gpt4_beats_gpt3", "gpt4_beats_gpt4_scm", "gpt3_beats_gpt3_scm"]]

In [83]:
lines = [
    "\\textbf{Conc}. & \\textbf{Prems} & \\textbf{Domain} &  \\human{\\textbf{H}} vs \\gptfour{\\textbf{4}}   & \\human{\\textbf{H}} vs \\gptthree{\\textbf{3}}  & \\gptfour{\\textbf{4}} vs \\gptthree{\\textbf{3}} & \\gptfour{\\textbf{4}} vs \\gptfourscm{\\textbf{4 SCM}} & \\gptthree{\\textbf{3}} vs \\gptthreescm{\\textbf{3 SCM}}\\\\",  
]

first = True
prow = None
for _, row in df.iterrows():
    
    line = ""
    if not first:
        if row["conclusion_type"] != prow["conclusion_type"] or row["num_premises"] != prow["num_premises"]:
            lines.append("\hline")
            
        for i, c in enumerate(["conclusion_type", "num_premises", "domain"]):
            if row["num_premises"] != prow["num_premises"] or c == "domain":
                line += f"{' ' if i > 0 else ''}{row[c]} &"
            else:
                line += f"{' ' if i > 0 else ''}&"
    else:
        lines.append("\hline")
        for i, c in enumerate(["conclusion_type", "num_premises", "domain"]):
            line += f"{' ' if i > 0 else ''}{row[c]} &"
    
    latex_name = lambda x: x.replace("_", "").replace("3", "three").replace("4", "four").replace("humanscm", "human").lower()
    t = len(df.columns[3:])
    bs, hl, hr = "\\", "{", "}"
    for i, comparison in enumerate(df.columns[3:]):
            c1,c2 = comparison.split("_beats_")
            if row[comparison] >= 0.95:
                line += f" {bs}{latex_name(c1)}{hl}{bs}textbf{hl}{row[comparison]}{hr}{hr}{' &' if i != t-1 else ''}"
            elif row[comparison] <= 0.05:
                line += f" {bs}{latex_name(c2)}{hl}{bs}textbf{hl}{row[comparison]}{hr}{hr}{' &' if i != t-1 else ''}"
            else:
                line += f" {row[comparison]}{' &' if i != t-1 else ''}"
    
    lines.append(line + f"{bs}{bs}")
    first = False
    prow = row

with open("../tables/table6.txt", "w") as f:
    for l in lines:
        f.write("%s\n" % l)